# Курсовая работа по машинному обучению: анализ и классификация данных Titanic

## 0. Введение

### Описание датасета
Датасет Titanic представляет собой исторические данные о пассажирах печально известного рейса Титаника. Целью анализа является предсказание выживаемости пассажиров на основе доступной информации, такой как возраст, пол, класс билета и другие характеристики.

**Источник данных:** [Kaggle Titanic Dataset](https://www.kaggle.com/c/titanic/data)

**Цель задачи:**
- Построить модель бинарной классификации для предсказания выживаемости пассажиров.

**Пример данных:**

```plaintext
| PassengerId | Survived | Pclass | Name              | Sex    | Age | SibSp | Parch | Ticket | Fare   | Cabin | Embarked |
|-------------|----------|--------|-------------------|--------|-----|-------|-------|--------|--------|-------|----------|
| 1           | 0        | 3      | Braund, Mr. Owen | male   | 22  | 1     | 0     | A/5 21171 | 7.25 | NaN   | S        |
| 2           | 1        | 1      | Cumings, Mrs. J  | female | 38  | 1     | 0     | PC 17599 | 71.28 | C85   | C        |
```


### API для загрузки и предварительного анализа данных

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

class TitanicDatasetLoader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = None

    def load_data(self):
        self.data = pd.read_csv(self.filepath)
        return self.data

    def get_sample(self, n=5):
        if self.data is None:
            raise ValueError("Данные не загружены. Используйте метод load_data().")
        return self.data.sample(n)

    def split_data(self, test_size=0.2, random_state=42):
        if self.data is None:
            raise ValueError("Данные не загружены. Используйте метод load_data().")
        X = self.data.drop(columns=['Survived'])
        y = self.data['Survived']
        return train_test_split(X, y, test_size=test_size, random_state=random_state)

# Пример использования
loader = TitanicDatasetLoader("titanic.csv")
data = loader.load_data()
print(loader.get_sample())

## 1. Исследование данных

### Статистика по данным



In [ ]:
# Основной анализ данных
print("Размер данных:", data.shape)
print("Количество пропусков:")
print(data.isnull().sum())

# Типы данных
print("Типы данных:")
print(data.dtypes)

# Распределение классов
print("Распределение классов выживаемости:")
print(data['Survived'].value_counts(normalize=True))

## 2. Выбор алгоритма

### Постановка задачи
Мы решаем задачу бинарной классификации: предсказать, выжил пассажир или нет (Survived).

### Пример алгоритма


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils import resample
import numpy as np

# Список моделей
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42),
    "SVM": SVC(random_state=42, probability=True),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

# Функция для бутстрап-оценки

def bootstrap_evaluation(model, X_train, y_train, X_test, y_test, n_iterations=100):
    scores = []
    for _ in range(n_iterations):
        X_resampled, y_resampled = resample(X_train, y_train)
        model.fit(X_resampled, y_resampled)
        scores.append(model.score(X_test, y_test))
    return np.mean(scores), np.std(scores)

# Сравнение моделей
for name, model in models.items():
    mean_score, std_score = bootstrap_evaluation(model, X_train, y_train, X_test, y_test)
    print(f"{name}: Точность = {mean_score:.3f} ± {std_score:.3f}")

### Удаление выбросов и повторное обучение


In [ ]:
from sklearn.ensemble import IsolationForest

# Удаление выбросов
iso_forest = IsolationForest(contamination=0.1, random_state=42)
outlier_mask = iso_forest.fit_predict(X_train) == 1
X_train_cleaned = X_train[outlier_mask]
y_train_cleaned = y_train[outlier_mask]

# Обучение модели на очищенных данных
model_cleaned = RandomForestClassifier(random_state=42)
model_cleaned.fit(X_train_cleaned, y_train_cleaned)

# Инференс
predictions_cleaned = model_cleaned.predict(X_test)
print("Точность модели после удаления выбросов:", accuracy_score(y_test, predictions_cleaned))

### Обучение на данных со сниженной размерностью

# Обучение модели на данных после PCA


In [ ]:
# Обучение модели на данных после PCA
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

model_pca = RandomForestClassifier(random_state=42)
model_pca.fit(X_train_pca, y_train)

predictions_pca = model_pca.predict(X_test_pca)
print("Точность модели на данных со сниженной размерностью:", accuracy_score(y_test, predictions_pca))

### Сравнение различных алгоритмов


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils import resample
import numpy as np

# Список моделей
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42),
    "SVM": SVC(random_state=42, probability=True),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

# Функция для бутстрап-оценки

def bootstrap_evaluation(model, X_train, y_train, X_test, y_test, n_iterations=100):
    scores = []
    for _ in range(n_iterations):
        X_resampled, y_resampled = resample(X_train, y_train)
        model.fit(X_resampled, y_resampled)
        scores.append(model.score(X_test, y_test))
    return np.mean(scores), np.std(scores)

# Сравнение моделей
for name, model in models.items():
    mean_score, std_score = bootstrap_evaluation(model, X_train, y_train, X_test, y_test)
    print(f"{name}: Точность = {mean_score:.3f} ± {std_score:.3f}")

## 3. Кластеризация и понижение размерности

### PCA для визуализации


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Понижение размерности
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_train)

plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_train, cmap='viridis')
plt.title('PCA визуализация данных')
plt.show()

## 4. Аномалии


In [ ]:
from sklearn.ensemble import IsolationForest

# Поиск аномалий
iso_forest = IsolationForest(contamination=0.1, random_state=42)
iso_forest.fit(X_train)
anomalies = iso_forest.predict(X_train)

print("Количество аномалий:", sum(anomalies == -1))

## 5. ML-systems design

### Возможности продакшн-системы
- Автоматическое предсказание выживаемости пассажиров на основе данных о рейсе.
- Пример использования: прогнозирование рисков для транспортных систем.

### Преимущества и недостатки
**Преимущества:**
- Простота использования.
- Доступность данных для обучения.

**Недостатки:**
- Ограниченность характеристик.
- Необъективность исторических данных.


## Заключение
Данная курсовая работа демонстрирует основные этапы анализа данных, выбора модели и подготовки системы для задач машинного обучения. Датасет Titanic является удобным инструментом для обучения, но имеет ограничения, которые необходимо учитывать при использовании в реальных системах.